In [1]:
!pip install geopy

In [2]:
from geopy.geocoders import Nominatim

In [3]:
geolocator = Nominatim(user_agent = 'kaggle_learn')
location = geolocator.geocode('Taj Mahal')

print(location.point)
print(location.address)

27 10m 30.027s N, 78 2m 31.5645s E
Taj Mahal, Taj East Gate Road, Taj Ganj, Agra, Agra District, Uttar Pradesh, 282004, India


In [5]:
#Getting the Latitude and Longitude
point = location.point
print('Latitude:', point.latitude)
print('Longitude', point.longitude)

Latitude: 27.1750075
Longitude 78.04210126365584


In [8]:
import pandas as pd
universities = pd.read_csv("data/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [9]:
#Using a lambda function to apply the geocoder to every row in the DataFrame.

In [23]:
import numpy as np
import pandas as pd
import geopandas as gpd  # Import geopandas
from geopy.geocoders import Nominatim  # Import Nominatim geocoder
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="my_geocoder")

def my_geocoder(row):
    try:
        location = geolocator.geocode(row, timeout=10)
        if location:
            return pd.Series({'Latitude': location.latitude, 'Longitude': location.longitude})
        else:
            return None
    except GeocoderTimedOut:
        return None

# Your universities DataFrame here

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(pd.isna(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.dropna(subset=["Latitude"])
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

100.0% of addresses were geocoded!


/Users/anishsoni/anaconda3/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Name,"(Latitude, Longitude)",Latitude,Longitude,geometry
0,University of Oxford,None,33.650290,-117.828179,POINT (-117.82818 33.65029)
1,University of Cambridge,None,52.210946,0.092005,POINT (0.09200 52.21095)
2,Imperial College London,None,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,None,47.376426,8.547671,POINT (8.54767 47.37643)
4,UCL,None,51.521785,-0.135151,POINT (-0.13515 51.52179)


In [28]:
!pip install folium
from folium import Marker
import folium

# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

In [49]:
#merging data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
asia = world.loc[world.continent == 'Asia'].reset_index(drop=True)

asia_stats = asia[["name", "pop_est", "gdp_md_est"]]
asia_boundaries = asia[["name", "geometry"]]

/var/folders/7f/7tqk6h_s4kj09qwljz25pppr0000gn/T/ipykernel_40884/2451058880.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [50]:
asia_boundaries.head()

,name,geometry
0,Kazakhstan,"POLYGON ((87.35997 49.21498, 86.59878 48.54918..."
1,Uzbekistan,"POLYGON ((55.96819 41.30864, 55.92892 44.99586..."
2,Indonesia,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ..."
3,Timor-Leste,"POLYGON ((124.96868 -8.89279, 125.08625 -8.656..."
4,Israel,"POLYGON ((35.71992 32.70919, 35.54567 32.39399..."


In [51]:
asia_stats.head()

,name,pop_est,gdp_md_est
0,Kazakhstan,18513930.0,181665
1,Uzbekistan,33580650.0,57921
2,Indonesia,270625568.0,1119190
3,Timor-Leste,1293119.0,2017
4,Israel,9053300.0,394652


In [52]:
#attribute join
europe = europe_boundaries.merge(asia_stats, on='name')
europe.head()

,name,geometry,pop_est,gdp_md_est


In [47]:
#SPATIAL JOIN
#With a spatial join, we combine GeoDataFrames based on the spatial relationship between the objects in the "geometry" columns. For instance, we already have a GeoDataFrame universities containing geocoded addresses of European universities.



We located 95 universities.
Only 0 of the universities were located in Europe (in 0 different countries).


/var/folders/7f/7tqk6h_s4kj09qwljz25pppr0000gn/T/ipykernel_40884/2355514550.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  european_universities = gpd.sjoin(universities, europe)


,Name,"(Latitude, Longitude)",Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
